**Trabalho 2 - Mineração de Dados Não Estruturados - Mineração de Imagens**

**Luiz Henrique Lourenção - 10284862** 

#Objetivo

O dataset se encontra neste link: https://www.kaggle.com/gpiosenka/100-bird-species

O dataset é uma coleção de cerca de 50 mil imagens de pássaros e tem como objetivo a criação de um modelo de classificação de espécies de pássaros.

O dataset conta com 325 espécies de pássaros. Todas as imagens são 224 x 224 x 3, coloridas e em formato .jpg.

Nestre trabalho, utilizarei algumas das tecnicas aprendidas em aula para tentar desenvolver um modelo que resolva o problema, que é acertar qual a espécie de um determinado pássaro.

#Configurações

In [1]:
import numpy as np
import pandas as pd
import cv2
from google.colab.patches import cv2_imshow # for image display
from skimage import io
from PIL import Image 
import matplotlib.pylab as plt
from os import listdir
from os.path import isfile, join
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import classification_report

In [2]:
#https://drive.google.com/file/d/1DkgHPCiSpv7rCjztvpY5gW0DVVJLQXuk/view?usp=sharing
!gdown --id 1DkgHPCiSpv7rCjztvpY5gW0DVVJLQXuk

Downloading...
From: https://drive.google.com/uc?id=1DkgHPCiSpv7rCjztvpY5gW0DVVJLQXuk
To: /content/passaros.zip
100% 1.13G/1.13G [00:06<00:00, 184MB/s]


In [3]:
!unzip passaros.zip

A saída de streaming foi truncada nas últimas 5000 linhas.
  inflating: train/TURQUOISE MOTMOT/043.jpg  
  inflating: train/TURQUOISE MOTMOT/044.jpg  
  inflating: train/TURQUOISE MOTMOT/045.jpg  
  inflating: train/TURQUOISE MOTMOT/046.jpg  
  inflating: train/TURQUOISE MOTMOT/047.jpg  
  inflating: train/TURQUOISE MOTMOT/048.jpg  
  inflating: train/TURQUOISE MOTMOT/049.jpg  
  inflating: train/TURQUOISE MOTMOT/050.jpg  
  inflating: train/TURQUOISE MOTMOT/051.jpg  
  inflating: train/TURQUOISE MOTMOT/052.jpg  
  inflating: train/TURQUOISE MOTMOT/053.jpg  
  inflating: train/TURQUOISE MOTMOT/054.jpg  
  inflating: train/TURQUOISE MOTMOT/055.jpg  
  inflating: train/TURQUOISE MOTMOT/056.jpg  
  inflating: train/TURQUOISE MOTMOT/057.jpg  
  inflating: train/TURQUOISE MOTMOT/058.jpg  
  inflating: train/TURQUOISE MOTMOT/059.jpg  
  inflating: train/TURQUOISE MOTMOT/060.jpg  
  inflating: train/TURQUOISE MOTMOT/061.jpg  
  inflating: train/TURQUOISE MOTMOT/062.jpg  
  inflating: train/TU

#Preparando o Dataset

In [4]:
df = pd.read_csv('birds.csv')
df

,filepaths,labels,data set
0,train/AFRICAN CROWNED CRANE/001.jpg,AFRICAN CROWNED CRANE,train
1,train/AFRICAN CROWNED CRANE/002.jpg,AFRICAN CROWNED CRANE,train
2,train/AFRICAN CROWNED CRANE/003.jpg,AFRICAN CROWNED CRANE,train
3,train/AFRICAN CROWNED CRANE/004.jpg,AFRICAN CROWNED CRANE,train
4,train/AFRICAN CROWNED CRANE/005.jpg,AFRICAN CROWNED CRANE,train
...,...,...,...
50577,valid/YELLOW HEADED BLACKBIRD/1.jpg,YELLOW HEADED BLACKBIRD,valid
50578,valid/YELLOW HEADED BLACKBIRD/2.jpg,YELLOW HEADED BLACKBIRD,valid
50579,valid/YELLOW HEADED BLACKBIRD/3.jpg,YELLOW HEADED BLACKBIRD,valid
50580,valid/YELLOW HEADED BLACKBIRD/4.jpg,YELLOW HEADED BLACKBIRD,valid


In [14]:
train_dataset = []
test_dataset = []
valid_dataset = []
train_y = []
test_y = []
valid_y = []
for img_url in df['filepaths']:
  try:
    image = io.imread(img_url)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    split_url = img_url.split("/")
    species = split_url[1]
    purpose = split_url[0]
    if purpose == 'train':
      train_dataset.append(image)
      train_y.append(species)
    elif purpose == 'test':
      test_dataset.append(image)
      test_y.append(species)
    else:
      valid_dataset.append(image)
      valid_y.append(species)
  except:
    print('Warning: ', img_url)

#Histogramas de Cor

##Extraindo e organizando os dados

In [6]:
color = ('b','g','r')

train_dataset_hist_r = []
train_dataset_hist_g = []
train_dataset_hist_b = []

counter = 0
for image in train_dataset:
  hists = {}
  for i,col in enumerate(color):
    histr = cv2.calcHist([image],[i],None,[256],[0,256])
    if col == 'r': train_dataset_hist_r.append(histr)
    if col == 'g': train_dataset_hist_g.append(histr)
    if col == 'b': train_dataset_hist_b.append(histr)

train_X_r = np.array(train_dataset_hist_r)
length = np.sqrt((train_X_r**2).sum(axis=1))[:,None]
train_X_r = train_X_r / length

train_X_g = np.array(train_dataset_hist_g)
length = np.sqrt((train_X_g**2).sum(axis=1))[:,None]
train_X_g = train_X_g / length

train_X_b = np.array(train_dataset_hist_b)
length = np.sqrt((train_X_b**2).sum(axis=1))[:,None]
train_X_b = train_X_b / length


test_dataset_hist_r = []
test_dataset_hist_g = []
test_dataset_hist_b = []

counter = 0
for image in test_dataset:
  hists = {}
  for i,col in enumerate(color):
    histr = cv2.calcHist([image],[i],None,[256],[0,256])
    if col == 'r': test_dataset_hist_r.append(histr)
    if col == 'g': test_dataset_hist_g.append(histr)
    if col == 'b': test_dataset_hist_b.append(histr)

test_X_r = np.array(test_dataset_hist_r)
length = np.sqrt((test_X_r**2).sum(axis=1))[:,None]
test_X_r = test_X_r / length

test_X_g = np.array(test_dataset_hist_g)
length = np.sqrt((test_X_g**2).sum(axis=1))[:,None]
test_X_g = test_X_g / length

test_X_b = np.array(test_dataset_hist_b)
length = np.sqrt((test_X_b**2).sum(axis=1))[:,None]
test_X_b = test_X_b / length


valid_dataset_hist_r = []
valid_dataset_hist_g = []
valid_dataset_hist_b = []

counter = 0
for image in valid_dataset:
  hists = {}
  for i,col in enumerate(color):
    histr = cv2.calcHist([image],[i],None,[256],[0,256])
    if col == 'r': valid_dataset_hist_r.append(histr)
    if col == 'g': valid_dataset_hist_g.append(histr)
    if col == 'b': valid_dataset_hist_b.append(histr)

valid_X_r = np.array(valid_dataset_hist_r)
length = np.sqrt((valid_X_r**2).sum(axis=1))[:,None]
valid_X_r = valid_X_r / length

valid_X_g = np.array(valid_dataset_hist_g)
length = np.sqrt((valid_X_g**2).sum(axis=1))[:,None]
valid_X_g = valid_X_g / length

valid_X_b = np.array(valid_dataset_hist_b)
length = np.sqrt((valid_X_b**2).sum(axis=1))[:,None]
valid_X_b = valid_X_b / length

In [7]:
train_X = np.concatenate((train_X_r,train_X_g,train_X_g),axis=1)
test_X = np.concatenate((test_X_r,test_X_g,test_X_g),axis=1)
valid_X = np.concatenate((valid_X_r,valid_X_g,valid_X_g),axis=1)
train_X.shape

(47332, 768, 1)

In [8]:
train_X = train_X.reshape(train_X.shape[0],train_X.shape[1])
test_X = test_X.reshape(test_X.shape[0],test_X.shape[1])
valid_X = valid_X.reshape(valid_X.shape[0],valid_X.shape[1])
train_X.shape

(47332, 768)

In [9]:
pd.DataFrame(train_X)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,728,729,730,731,732,733,734,735,736,737,738,739,740,741,742,743,744,745,746,747,748,749,750,751,752,753,754,755,756,757,758,759,760,761,762,763,764,765,766,767
0,0.000000,0.000000,0.000000,0.000266,0.000000,0.000000,0.000532,0.000532,0.000532,0.000799,0.001331,0.000799,0.002129,0.002396,0.005323,0.003726,0.003460,0.004525,0.010647,0.006388,0.011179,0.012776,0.011445,0.013575,0.008784,0.011445,0.017833,0.013841,0.016236,0.016236,0.015704,0.016502,0.015970,0.014905,0.014373,0.019430,0.022092,0.025552,0.024754,0.028746,...,0.022681,0.034421,0.032553,0.028551,0.026683,0.026950,0.029885,0.025082,0.026683,0.030685,0.031219,0.023481,0.024015,0.023748,0.025349,0.025082,0.026416,0.024015,0.026950,0.021080,0.021080,0.022414,0.019745,0.021613,0.023481,0.022681,0.021080,0.020546,0.020012,0.022947,0.026149,0.023748,0.024548,0.024815,0.026683,0.038957,0.033621,0.032286,0.029084,0.040558
1,0.000000,0.000000,0.000227,0.000454,0.000000,0.000454,0.000454,0.000227,0.001362,0.002269,0.003177,0.003177,0.009304,0.009531,0.011800,0.018381,0.023147,0.035174,0.034720,0.036762,0.041755,0.049470,0.050378,0.047655,0.058774,0.061043,0.059682,0.056051,0.060363,0.053782,0.056959,0.059909,0.058093,0.062178,0.054236,0.051286,0.055597,0.046293,0.047882,0.047882,...,0.003800,0.003167,0.004223,0.002745,0.003378,0.002745,0.003589,0.002534,0.002322,0.002956,0.001689,0.002322,0.002745,0.001900,0.000422,0.000633,0.000845,0.001267,0.001056,0.000845,0.001689,0.003167,0.001689,0.002745,0.002322,0.001478,0.002745,0.001056,0.001689,0.001900,0.000845,0.000845,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,0.046653,0.034390,0.027458,0.022393,0.026126,0.023460,0.018928,0.011996,0.011996,0.014396,0.012263,0.015462,0.009597,0.010663,0.012796,0.010930,0.012530,0.012530,0.014129,0.010930,0.015462,0.013863,0.014662,0.014662,0.014929,0.015729,0.016262,0.019461,0.017595,0.021327,0.016262,0.024526,0.022127,0.025326,0.025326,0.030124,0.034123,0.031457,0.032524,0.041854,...,0.001212,0.002182,0.001697,0.002909,0.002182,0.002425,0.002182,0.002425,0.002425,0.002667,0.002909,0.001697,0.002909,0.003152,0.002909,0.002425,0.002667,0.001940,0.002909,0.000970,0.001697,0.001697,0.000485,0.000485,0.000242,0.000242,0.000727,0.000242,0.000242,0.000242,0.000000,0.000242,0.000727,0.000000,0.000485,0.000485,0.000242,0.000485,0.000485,0.000242
3,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000634,0.000211,0.000634,0.000211,0.001691,0.000634,0.001268,0.001268,0.001480,0.002114,0.003171,0.003805,0.006976,0.007187,0.010358,0.012894,0.010992,0.012471,0.015642,0.017545,0.020715,0.019447,0.026634,0.019870,0.021561,0.027479,0.026634,0.025577,0.028536,0.027268,0.023886,0.030016,...,0.001391,0.000795,0.002384,0.001987,0.001192,0.001192,0.000596,0.002186,0.000795,0.001987,0.001391,0.000397,0.000397,0.000397,0.000993,0.000199,0.000397,0.000199,0.000397,0.000199,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4,0.000000,0.000216,0.000216,0.000647,0.000431,0.001294,0.001294,0.004097,0.005822,0.008410,0.007331,0.008841,0.006685,0.009272,0.008841,0.009919,0.013800,0.012722,0.010997,0.013153,0.015310,0.012722,0.018113,0.012722,0.014232,0.015094,0.014879,0.015957,0.016172,0.020054,0.020269,0.021779,0.019407,0.020054,0.019622,0.023073,0.017250,0.016388,0.019622,0.025013,...,0.012866,0.012233,0.011178,0.014553,0.011178,0.016030,0.018771,0.014975,0.021513,0.012655,0.014553,0.013077,0.011178,0.012866,0.007593,0.007804,0.005062,0.007593,0.004640,0.003164,0.005695,0.002109,0.001898,0.001476,0.001265,0.001687,0.001265,0.000422,0.000844,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...

##Classificação - Árvore de Decisão

In [10]:
model_tree_color = DecisionTreeClassifier(random_state=5)

model_tree_color.fit(train_X, train_y)

tree_color_pred = model_tree_color.predict(test_X)

In [11]:
print(classification_report(test_y, tree_color_pred))

                             precision    recall  f1-score   support

      AFRICAN CROWNED CRANE       0.00      0.00      0.00         5
          AFRICAN FIREFINCH       0.00      0.00      0.00         5
                  ALBATROSS       0.00      0.00      0.00         5
       ALEXANDRINE PARAKEET       0.25      0.20      0.22         5
            AMERICAN AVOCET       0.00      0.00      0.00         5
           AMERICAN BITTERN       0.00      0.00      0.00         5
              AMERICAN COOT       0.00      0.00      0.00         5
         AMERICAN GOLDFINCH       0.67      0.40      0.50         5
           AMERICAN KESTREL       0.00      0.00      0.00         5
             AMERICAN PIPIT       0.00      0.00      0.00         5
          AMERICAN REDSTART       0.00      0.00      0.00         5
                    ANHINGA       0.00      0.00      0.00         5
          ANNAS HUMMINGBIRD       0.00      0.00      0.00         5
                    ANTBIRD      

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


#Modelos Pré Treinados

##Configurações (ResNet18)

In [5]:
import torch
import torch.nn as nn
import torchvision.models as models
import torchvision.transforms as transforms
from torch.autograd import Variable
from PIL import Image

In [6]:
model = models.resnet18(pretrained=True)

layer = model._modules.get('avgpool')
model.eval() # coloca a rede em model eval() para desativar as dropout layers (sao uteis durante o treinamento apenas)

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

##Extração das Embeddings

In [7]:
scaler = transforms.Scale((224, 224))
normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
to_tensor = transforms.ToTensor()

/usr/local/lib/python3.7/dist-packages/torchvision/transforms/transforms.py:317: UserWarning: The use of the transforms.Scale transform is deprecated, please use transforms.Resize instead.
  warnings.warn("The use of the transforms.Scale transform is deprecated, " +


In [8]:
def gerar_embeddings(arquivo_imagem):
    
    img = Image.open(arquivo_imagem)
    img_transformada = Variable(normalize(to_tensor(scaler(img))).unsqueeze(0))

    embedding = torch.zeros(512) # armazenado espaço

    def capturar_embedding(m, i, o):
        embedding.copy_(o.data.reshape(o.data.size(1)))
    
    # capturando embeddings
    h = layer.register_forward_hook(capturar_embedding) 
    model(img_transformada)
    h.remove()
    
    return np.array(embedding)

Aqui eu realizei a extração das embeddings apenas dos datasets de teste e de validação, pois demora muito para rodar e o dataset de treino tem um número muito elevado de imagens.

Nas células abaixo, "test_embedding" se comporta como "train_X" e "valid_embedding" se comporta como "test_X", as respectivas colunas de target se comportam de acordo.

In [9]:
test_embedding = []
valid_embedding = []
for img_url in df['filepaths']:
  try:
    split_url = img_url.split("/")
    purpose = split_url[0]
    if purpose == 'train':
      continue
    elif purpose == 'test':
      embedding = gerar_embeddings(img_url)
      test_embedding.append(embedding)
    else:
      embedding = gerar_embeddings(img_url)
      valid_embedding.append(embedding)
  except:
    print('Warning: ', img_url)

In [15]:
model_tree_embedding = DecisionTreeClassifier(random_state=5)

model_tree_embedding.fit(test_embedding, test_y)

tree_embedding_pred = model_tree_embedding.predict(valid_embedding)

In [16]:
print(classification_report(valid_y, tree_embedding_pred))

                             precision    recall  f1-score   support

      AFRICAN CROWNED CRANE       0.50      0.40      0.44         5
          AFRICAN FIREFINCH       0.50      0.20      0.29         5
                  ALBATROSS       0.10      0.20      0.13         5
       ALEXANDRINE PARAKEET       0.75      0.60      0.67         5
            AMERICAN AVOCET       0.00      0.00      0.00         5
           AMERICAN BITTERN       0.67      0.40      0.50         5
              AMERICAN COOT       0.40      0.40      0.40         5
         AMERICAN GOLDFINCH       0.33      0.20      0.25         5
           AMERICAN KESTREL       0.33      0.20      0.25         5
             AMERICAN PIPIT       0.00      0.00      0.00         5
          AMERICAN REDSTART       0.00      0.00      0.00         5
                    ANHINGA       0.25      0.20      0.22         5
          ANNAS HUMMINGBIRD       0.00      0.00      0.00         5
                    ANTBIRD      

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


#Uso do Conhecimento

Este modelo pode ser utilizado por pesquisadores para auxiliar no reconhecimento das espécies de pássaros encontrados por eles.

#Conclusões

A acurácia dos modelos feitos nesse trabalho pode parecer pequena, porém se trata de um dataset com 325 possibilidades no "target".

Por exemplo, vamos considerar um classificador totalmente aleatório. A chance de acerto dele seria por volta de 1/325, o que resultaria em uma taxa de acerto próxima de 0.3%. O primeiro modelo (Histogramas de Cor) conseguiu acertar 4%, que é 13 vezes maior do que seria um classificador randomico. O modelo baseado na extração de embeddings utilizando a rede pré treinada ResNet18 se saiu ainda melhor, conseguindo acertar 15% dos casos, que é 50 vezes melhor do que uma classificação completamente randomica.

Podemos então concluir que os modelos feitos nesse trabalho se saíram bem, levando em consideração o número de classes possíveis. Porém, com certeza existem modelos mais elaborados que conseguem atingir uma acurácia muito mais elevada.